Este colab fue desarrollado por Arnold Charry Armero.

# Huntco Company

## Conjuntos

$ i = \text{Planta indexada por }i$

$ j = \text{Almacén indexado por }j$

$ k = \text{Cliente indexado por }k$

## Parámetros

$ C_{i} = \textrm{Capacidad de la planta } i$

$ M = \textrm{Número muy grande para la capacidad ilimitada del almacén } j$

$ CP_{ij} = \textrm{Costo de producir salsa de tomate en la planta } i \textrm{ y transportarla al almacén }j$

$ CT_{jk} = \textrm{Costo de transportar la salsa del almacén } j \textrm{ al cliente }k$

$ CF_{i} = \textrm{Costo fijo de tener una planta } i$

$ CFA_{j} = \textrm{Costo fijo de tener un almacén } j$

$ D_{k} = \textrm{Demanda del cliente } k$

## Variables de decisión

$ x_{ij} = \textrm{Unidades que se producen en la planta } i \textrm{ y se transportan al almacén }j$

$ t_{jk} = \textrm{Unidades que se transportan del almacén } j \textrm{ y se transportan al cliente }k$

$ y_{i} = \textrm{1 se abre la planta } i \textrm{, 0 lo contrario} $

$ w_{j} = \textrm{1 se abre el almacén } j \textrm{, 0 lo contrario} $

## Función Objetivo

$$\textrm{Min } z = \sum_{j = 1}^{n}\sum_{i = 1}^{n} CP_{ij}\: x_{ij} + \sum_{i = 1}^{n} CF_{i}\: y_{i} + \sum_{k = 1}^{n}\sum_{j = 1}^{n} CT_{jk}\: t_{jk} + \sum_{j = 1}^{n} CFA_{j}\: w_{j}$$

## Restricciones

No se puede rebasar la capacidad de cada planta,

$$ \sum_{j = 1}^{n} x_{ij} \leq C_{i}\: y_{i}, \forall i$$

Lo que sale de la planta, entra al almacén


$$ \sum_{i = 1}^{n} x_{ij} = \sum_{k = 1}^{n} t_{jk}, \forall j$$

Lo que sale de los almacénes, debe ser para satisfacer al cliente,

$$ \sum_{j = 1}^{n} t_{jk} \geq D_{k}, \forall k$$

Cada almacén tiene una capacidad ilimitada,

$$ \sum_{k = 1}^{n} t_{jk} \leq M w_{j}, \forall j$$

In [ ]:
#Se instala la libreria
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 20.5 MB/s eta 0:00:00


In [ ]:
#Se importa la libreria
from pulp import *

In [ ]:
#Conjuntos
plantas = range(5)
almacén = range(3)
clientes = range(4)

In [ ]:
#Parámetros

C = [300, 200, 300, 200, 400]

CP = [[8, 10, 12],
      [7, 5, 7],
      [8, 6, 5],
      [5, 6, 7],
      [7, 6, 5]]

CT = [[40, 80, 90, 50],
      [70, 70, 60, 80],
      [80, 30, 50, 60]]

D = [200, 300, 150, 250]

CF = [35000, 45000, 40000, 42000, 40000]

CFA = [30000, 40000, 30000]

M = 10000000

In [ ]:
#Se crea la instancia del modelo
model = LpProblem(name = "Huntco_Company", sense = LpMinimize)

#Inicialización de las variables de decisión
X = LpVariable.dicts("X", (plantas, almacén), 0, None, LpInteger)
Y = LpVariable.dicts("Y", plantas, 0, 1, LpInteger)
W = LpVariable.dicts("W", almacén, 0, 1, LpInteger)
T = LpVariable.dicts("T", (almacén, clientes), 0, None, LpInteger)


#Se agrega la función objetivo
model += (lpSum([CP[i][j] * X[i][j] for i in plantas for j in almacén]) + lpSum([CF[i] * Y[i] for i in plantas]) + lpSum([CT[j][k] * T[j][k] for j in almacén for k in clientes]) + lpSum([CFA[j] * W[j] for j in almacén]))

#Restricciones

#No se debe sobrepasar la capacidad
for i in plantas:
  model += lpSum([X[i][j] for j in almacén]) <= C[i] * Y[i]

#Lo que entra a cada almacen, sale del almacen
for j in almacén:
  model += lpSum([X[i][j] for i in plantas]) == lpSum([T[j][k] for k in clientes])

#Lo que sale de los almacénes, debe ser para satisfacer al cliente
for k in clientes:
  model += lpSum([T[j][k] for j in almacén]) >= D[k]

#Cada almacén tiene una capacidad
for j in almacén:
  model += lpSum([T[j][k] for k in clientes]) <= M * W[j]

#Se resuelve el modelo
model.solve()

#Se imprime el status del problema
print("Status", LpStatus[model.status])

#Se imprimen las variables
for v in model.variables():
  print(v.name, "=", v.varValue)

#Se imprime la función objetivo
print("Costos Totales = $", value(model.objective))

Status Optimal
T_0_0 = 0.0
T_0_1 = 0.0
T_0_2 = 0.0
T_0_3 = 0.0
T_1_0 = 0.0
T_1_1 = 0.0
T_1_2 = 0.0
T_1_3 = 0.0
T_2_0 = 200.0
T_2_1 = 300.0
T_2_2 = 150.0
T_2_3 = 250.0
W_0 = 0.0
W_1 = 0.0
W_2 = 1.0
X_0_0 = 0.0
X_0_1 = 0.0
X_0_2 = 200.0
X_1_0 = 0.0
X_1_1 = 0.0
X_1_2 = 0.0
X_2_0 = 0.0
X_2_1 = 0.0
X_2_2 = 300.0
X_3_0 = 0.0
X_3_1 = 0.0
X_3_2 = 0.0
X_4_0 = 0.0
X_4_1 = 0.0
X_4_2 = 400.0
Y_0 = 1.0
Y_1 = 0.0
Y_2 = 1.0
Y_3 = 0.0
Y_4 = 1.0
Costos Totales = $ 198400.0
